# Animation example

In [1]:
#@title Mount Google Drive
try:
    from google.colab import drive
    drive.mount('/content/gdrive')
    outputs_path = "/content/gdrive/MyDrive/AI/StabilityAnimations"
    !mkdir -p $outputs_path
except:
    outputs_path = "."
print(f"Animations will be saved to {outputs_path}")

In [2]:
#@title Connect to the Stability API
import grpc
import os
from pathlib import Path
import shutil
import sys


path = Path('stability-sdk')
if path.exists():
    shutil.rmtree(path)
    !pip uninstall -y stability-sdk


!git clone -b anima --recurse-submodules https://github.com/Stability-AI/stability-sdk
!touch ./stability-sdk/src/stability_sdk/interfaces/__init__.py
!pip install ./stability-sdk


from stability_sdk import client
from stability_sdk.utils import (
    color_match_from_string,
    sampler_from_string,
    key_frame_inbetweens,
    key_frame_parse,
    guidance_from_string,
    #curve_to_series,
    image_mix,
    image_to_jpg_bytes,
    image_to_png_bytes,
    image_to_prompt,
    image_xform,
    warp2d_op,
    warp3d_op,
    border_mode_from_str_2d,
    border_mode_from_str_3d,

)


from stability_sdk.client import generation, generation_grpc # not a huge fan of this but at least it works


# GRPC endpoint and engines
GRPC_HOST = "" #@param {type:"string"}
API_KEY = "" #@param {type:"string"}
GENERATE_ENGINE_ID = 'stable-diffusion-v1-5'
INPAINT_ENGINE_ID = 'stable-diffusion-v1-5'
TRANSFORM_ENGINE_ID = 'transform-server-v1'


# Connect to Stability API
stub = client.open_channel(GRPC_HOST, api_key=API_KEY)


In [3]:
#@title Code definitions

import bisect
import cv2
import datetime
import json
import logging
import numpy as np
import os
import pandas as pd
import pathlib
import random
import re
import subprocess
import sys

from base64 import b64encode
from collections import OrderedDict
from IPython import display
from PIL import Image
from tqdm import tqdm
from types import SimpleNamespace
from typing import List, Tuple




In [4]:
#@title Settings

def Args():

    #@markdown ####**Settings:**
    W = 512 #@param
    H = 512 #@param
    W, H = map(lambda x: x - x % 64, (W, H))  # resize to integer multiple of 64
    sampler = 'K_euler_ancestral' #@param ["DDIM", "PLMS", "K_euler", "K_euler_ancestral", "K_heun", "K_dpm_2", "K_dpm_2_ancestral", "K_lms"]
    seed = -1 #@param
    cfg_scale = 7 #@param {type:"number"}
    clip_guidance = 'FastBlue' #@param ["None", "Simple", "FastBlue", "FastGreen"]

    #@markdown ####**Animation Settings:**
    animation_mode = '3D' #@param ['2D', '3D', 'Video Input'] {type:'string'}
    max_frames = 60 #@param {type:"number"}
    border = 'replicate' #@param ['reflect', 'replicate', 'wrap', 'zero'] {type:'string'}
    inpaint_border = False #@param {type:"boolean"}
    interpolate_prompts = False #@param {type:"boolean"}
    locked_seed = False #@param {type:"boolean"}

    #@markdown ####**Key framed value curves:**
    angle = "0:(1)" #@param {type:"string"}
    zoom = "0:(1.05)" #@param {type:"string"}
    translation_x = "0:(0)" #@param {type:"string"}
    translation_y = "0:(0)" #@param {type:"string"}
    translation_z = "0:(5)" #@param {type:"string"}
    rotation_x = "0:(0)" #@param {type:"string"}
    rotation_y = "0:(0)" #@param {type:"string"}
    rotation_z = "0:(1)" #@param {type:"string"}
    brightness_curve = "0: (1.0)" #@param {type:"string"}
    contrast_curve = "0: (1.0)" #@param {type:"string"}
    noise_curve = "0:(0.0)" # likely to be removed, still hidden here for potential experiments
    noise_scale_curve = "0:(1.02)" #@param {type:"string"}
    steps_curve = "0:(50)" #@param {type:"string"}
    strength_curve = "0:(0.65)" #@param {type:"string"}

    #@markdown ####**Coherence:**
    color_coherence = 'LAB' #@param ['None', 'HSV', 'LAB', 'RGB'] {type:'string'}
    diffusion_cadence_curve = "0:(4)" #@param {type:"string"}

    #@markdown ####**3D Depth Warping:**
    #use_depth_warping = True #@param {type:"boolean"}
    midas_weight = 0.3 #@param {type:"number"}
    near_plane = 200
    far_plane = 10000
    fov_curve = "0:(25)" #@param {type:"string"}
    save_depth_maps = False #@param {type:"boolean"}

    #@markdown ####**Video Input:**
    video_init_path = '/content/video_in.mp4' #@param {type:"string"}
    extract_nth_frame = 4 #@param {type:"number"}
    video_mix_in_curve = "0:(0.02)" #@param {type:"string"}
    video_flow_warp = True #@param {type:"boolean"}

    return locals()


### Prompts

In [5]:
animation_prompts = {
    0: "a painting of a delicious cheeseburger by Tyler Edlin",
    24: "a painting of the the answer to life the universe and everything by Tyler Edlin",
}

negative_prompt = ""
negative_prompt_weight = -1.0

#####################





In [6]:
#@title Render the animation

from stability_sdk.animation import Animator


# create folder for frames output
timestring = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
out_dir = os.path.join(outputs_path, timestring)
os.makedirs(out_dir, exist_ok=True)
print(f"Saving animation frames to {out_dir}...")

args = SimpleNamespace(**Args())
artist = Animator(args, out_dir)
artist.save_settings()
artist.setup_animation()
artist.render_animation(stub=stub)

In [57]:
#@title Create video from frames
skip_video_for_run_all = False #@param {type: 'boolean'}
fps = 12 #@param {type:"number"}

if skip_video_for_run_all == True:
    print('Skipping video creation, uncheck skip_video_for_run_all if you want to run it')
else:
    image_path = os.path.join(out_dir, "frame_%05d.png")
    mp4_path = os.path.join(out_dir, f"{timestring}.mp4")

    print(f"Compiling animation frames to {mp4_path}...")

    cmd = [
        'ffmpeg',
        '-y',
        '-vcodec', 'png',
        '-r', str(fps),
        '-start_number', str(0),
        '-i', image_path,
        '-c:v', 'libx264',
        '-vf',
        f'fps={fps}',
        '-pix_fmt', 'yuv420p',
        '-crf', '17',
        '-preset', 'veryfast',
        mp4_path
    ]
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    if process.returncode != 0:
        print(stderr)
        raise RuntimeError(stderr)

    mp4 = open(mp4_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display.display( display.HTML(f'<video controls loop><source src="{data_url}" type="video/mp4"></video>') )
